# Swift Identifier Extractor - RunPod Quick Start

이 노트북은 RunPod GPU 환경에서 Swift 식별자를 추출하는 빠른 시작 가이드입니다.

## 1. 환경 설정

In [ ]:
# 환경 설정 스크립트 실행
!bash setup.sh

## 2. CUDA 확인

In [ ]:
# NVIDIA GPU 정보
!nvidia-smi

## 3. 파일 확인

In [ ]:
import os
import json

# 필수 파일 확인
required_files = [
    'models/base_model.gguf',
    'models/lora.gguf',
    'dataset.jsonl'
]

print("필수 파일 확인:")
for file in required_files:
    exists = "✅" if os.path.exists(file) else "❌"
    print(f"{exists} {file}")

# 데이터셋 크기 및 구조 확인
if os.path.exists('dataset.jsonl'):
    with open('dataset.jsonl', 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        line_count = len(lines)
        
    print(f"\n데이터셋: {line_count}개 항목")
    
    # 첫 번째 항목 구조 확인
    if lines:
        first_item = json.loads(lines[0])
        print("\n데이터셋 구조:")
        for key in first_item.keys():
            print(f"  - {key}")
        
        # file_path 예시
        if 'file_path' in first_item:
            print(f"\n예시 file_path: {first_item['file_path']}")

## 4. 추론 실행 (기본 설정)

In [ ]:
# 기본 설정으로 실행
!python run_inference.py

## 5. 커스텀 설정으로 실행 (선택사항)

In [ ]:
# 커스텀 설정 예시
!python run_inference.py \
    --dataset dataset.jsonl \
    --base_model models/base_model.gguf \
    --lora models/lora.gguf \
    --output output/identifiers.txt \
    --ctx 8192 \
    --gpu_layers -1

## 6. 결과 확인

In [ ]:
import json

# 요약 정보 출력
with open('output/identifiers_summary.json', 'r') as f:
    summary = json.load(f)

print("=" * 50)
print("결과 요약")
print("=" * 50)
print(f"처리된 항목: {summary['processed_items']}개")
print(f"총 식별자: {summary['total_identifiers']}개")
print(f"고유 식별자: {summary['unique_identifiers']}개")
print(f"출력 파일: {summary['output_file']}")

# 식별자 샘플 출력
print("\n식별자 샘플 (상위 20개):")
print("-" * 50)
with open('output/identifiers.txt', 'r') as f:
    for i, line in enumerate(f):
        if i >= 20:
            break
        print(line.strip())

## 7. 체크포인트 확인

In [ ]:
import os

# 처리 완료된 파일 확인
if os.path.exists('checkpoint/processed.txt'):
    with open('checkpoint/processed.txt', 'r') as f:
        processed = [line.strip() for line in f if line.strip()]
    print(f"처리 완료: {len(processed)}개 파일")
    
    # 처음 5개 경로 출력
    if processed:
        print("\n예시 (처음 5개):")
        for path in processed[:5]:
            print(f"  ✓ {path}")
else:
    print("아직 처리된 파일이 없습니다.")

## 8. 체크포인트 초기화 (처음부터 다시 시작)

In [ ]:
# 체크포인트 초기화
!python run_inference.py --reset

## 9. 결과 다운로드 준비

In [ ]:
# 결과 파일 압축
!zip -r results.zip output/ checkpoint/
print("\n✅ results.zip 생성 완료!")
print("파일 탐색기에서 다운로드하세요.")

## 10. 데이터셋 통계 (추가 분석)

In [ ]:
import json
from collections import Counter
import os

# 데이터셋 분석
print("데이터셋 상세 통계")
print("=" * 50)

# 레포지토리별 파일 수 집계
repo_counts = Counter()
total_size = 0

with open('dataset.jsonl', 'r') as f:
    for line in f:
        if line.strip():
            item = json.loads(line)
            file_path = item['file_path']
            
            # 레포지토리 이름 추출 (첫 번째 디렉토리)
            repo = file_path.split('/')[0]
            repo_counts[repo] += 1
            
            # 코드 길이
            total_size += len(item['content'])

print(f"총 파일 수: {sum(repo_counts.values())}개")
print(f"총 코드 크기: {total_size:,} bytes")
print(f"레포지토리 수: {len(repo_counts)}개")
print("\n레포지토리별 파일 수 (상위 10개):")

for repo, count in repo_counts.most_common(10):
    print(f"  {repo}: {count}개")